In [1]:
import ee
import geemap


In [3]:
# Initialize GEE
ee.Authenticate()
ee.Initialize(project = 'ee-lavibas23')

In [18]:
# Define approximate bounding box for Mahaweli Basin
mahaweli = ee.Geometry.Rectangle([79.9, 7.1, 81.3, 8.5])

# Create map and add layers
Map = geemap.Map(basemap='SATELLITE')
Map.addLayer(mahaweli, {'color': 'red'}, 'Mahaweli Bounding Box')
Map.centerObject(mahaweli, zoom=8)

Map


Map(center=[7.800184306805551, 80.59999999999985], controls=(WidgetControl(options=['position', 'transparent_b…

In [19]:
# Define date range
start_date = '2023-01-01'
end_date = '2023-12-31'

# CHIRPS Daily Rainfall
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
    .filterDate(start_date, end_date) \
    .filterBounds(mahaweli) \
    .select('precipitation')


In [20]:
# MODIS MOD16A2 (8-day ET in kg/m2/8days ~ mm/8days)
modis_et = ee.ImageCollection('MODIS/006/MOD16A2') \
    .filterDate(start_date, end_date) \
    .filterBounds(mahaweli) \
    .select('ET')


In [21]:
# Function to aggregate daily ET from 8-day products (approximate)
def et_daily(image):
    return image.divide(8).copyProperties(image, image.propertyNames())

modis_et_daily = modis_et.map(et_daily)


In [22]:
# Function to reduce each image over basin and get mean
def extract_mean(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=mahaweli,
        scale=500,
        maxPixels=1e9
    )
    date = image.date().format('YYYY-MM-dd')
    return ee.Feature(None, {'date': date, 'value': stats.values().get(0)})


In [23]:
# Extract time series for rainfall
rainfall_ts = chirps.map(extract_mean).filter(ee.Filter.notNull(['value']))
# Extract time series for ET
et_ts = modis_et_daily.map(extract_mean).filter(ee.Filter.notNull(['value']))


In [24]:
# Convert to Pandas dataframes
def get_timeseries(fc):
    features = fc.getInfo()['features']
    data = [(f['properties']['date'], f['properties']['value']) for f in features]
    import pandas as pd
    df = pd.DataFrame(data, columns=['date', 'value'])
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'])
    return df

rainfall_df = get_timeseries(rainfall_ts)
et_df = get_timeseries(et_ts)


In [25]:
# Show first rows
print("Rainfall Data")
print(rainfall_df.head())
print("\nET Data")
print(et_df.head())


Rainfall Data
        date     value
0 2023-01-01  0.000000
1 2023-01-02  0.013173
2 2023-01-03  0.000000
3 2023-01-04  0.000000
4 2023-01-05  1.291538

ET Data
        date      value
0 2023-01-01  39.935742
1 2023-01-09  36.477999
2 2023-01-17  27.601966
3 2023-01-25  37.514414
4 2023-02-02  36.065766


In [26]:
rainfall_df.to_csv('mahaweli_rainfall_2023.csv', index=False)
et_df.to_csv('mahaweli_et_2023.csv', index=False)
